# Sembrando éxito: cómo el aprendizaje automático ayuda a los agricultores a seleccionar los mejores cultivos

![Farmer in a field](farmer_in_a_field.jpg)

Medir métricas esenciales del suelo, como los niveles de nitrógeno, fósforo, potasio y el valor del pH, es un aspecto importante de la evaluación del estado del suelo. Sin embargo, puede ser un proceso costoso y que requiere mucho tiempo, lo que puede hacer que los agricultores prioricen qué métricas medir en función de sus limitaciones presupuestarias.

Los agricultores tienen varias opciones a la hora de decidir qué cultivo plantar en cada temporada. Su principal objetivo es maximizar el rendimiento de sus cultivos, teniendo en cuenta diferentes factores. Un factor crucial que afecta el crecimiento de los cultivos es la condición del suelo en el campo, que puede evaluarse midiendo elementos básicos como los niveles de nitrógeno y potasio. Cada cultivo tiene una condición de suelo ideal que garantiza un crecimiento óptimo y un rendimiento máximo.

Un agricultor se acercó a usted como experto en aprendizaje automático para pedirle ayuda para seleccionar el mejor cultivo para su campo. Le proporcionaron un conjunto de datos llamado `soil_measures.csv`, que contiene:

- `"N"`: Relación de contenido de nitrógeno en el suelo
- `"P"`: Relación de contenido de fósforo en el suelo
- `"K"`: Relación de contenido de potasio en el suelo.
- Valor `"pH"` del suelo
- `"crop"`: valores categóricos que contienen varios cultivos (variable objetivo).

Cada fila de este conjunto de datos representa varias medidas del suelo en un campo en particular. Según estas mediciones, el cultivo especificado en la columna "cultivo" es la opción óptima para ese campo.

En este proyecto, aplicará el aprendizaje automático para construir un modelo de clasificación de clases múltiples para predecir el tipo de "cultivo", mientras utiliza técnicas para evitar la multicolinealidad, que es un concepto en el que dos o más características están altamente correlacionadas.

In [4]:
# All required libraries are imported here for you.
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import f1_score

# Load the dataset
crops = pd.read_csv("soil_measures.csv")

# Write your code here